Repeat 2_analyze_TM_octahedra.ipynb, but include the entire commensurate MAGNDATA dataset (including crystallographic multiples) to demonstrate TM_oct trends are not function of chosen subset

In [1]:
import json
from monty.json import MontyDecoder
import os
import plotly.graph_objects as go
import plotly.express as px
from pymatgen.core import Element

from utils_kga.statistical_analysis.get_spin_and_bond_angle_statistics import *
from utils_kga.statistical_analysis.ks_test import compute_ks_weighted
from utils_kga.general import pretty_plot

In [2]:
# Load edge-df
with open("data/dfs_of_magnetic_edge_information_include_crystallographic_multiples.json") as f:
    dict_all_stats = json.load(f)
all_stats = {key: pd.DataFrame.from_dict(df) for key, df in dict_all_stats.items()}

# For metadata filtering
with open("../../data_retrieval_and_preprocessing_MAGNDATA/data/df_grouped_and_chosen_commensurate_MAGNDATA.json") as f:
    df = json.load(f, cls=MontyDecoder)

In [3]:
# Add is_tm bool for later easier analysis
for ang_df in all_stats.values():
    ang_df["site_is_tm"] = ang_df["site_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["site_to_is_tm"] = ang_df["site_to_element"].apply(lambda el: Element(el).is_transition_metal)
    ang_df["ligand_el_set"] = ang_df["ligand_elements"].apply(lambda ls: set(ls))

In [4]:
description = [
    "all edges with TM octahedra at both nodes",
    "all oxygen edges with TM octahedra at both nodes",
]
plot_dir = "plots/TM_octahedra_analysis_include_crystallographic_multiples/"
os.makedirs(plot_dir, exist_ok=True)
one_d_hist_file = "plots/TM_octahedra_analysis_include_crystallographic_multiples/1d_histograms_bond_angle_occurrences_as_f_spin_angle.html"
one_d_hist_inverse_file = "plots/TM_octahedra_analysis_include_crystallographic_multiples/1d_histograms_spin_angle_occurrences_as_f_bond_angle.html"
two_d_hist_file = "plots/TM_octahedra_analysis_include_crystallographic_multiples/2d_histograms_bond_and_spin_angle_occurrences.html"

# List of dicts for later evaluation of important occurrence peaks in subdatasets
special_occus = []

In [5]:
write_mode = "w"
for normalize_bool, normalize_string in zip([True, False], ["normalized occurrences", "absolute occurrences"]):
    for ligand_multiplicity_bool, ligand_multiplicity_string in zip([True, False], ["ligand multiplicity included", "no ligand multiplicity included"]):
        for data_string in description:
            entries = 0
            all_spin_occus = []
            for md_id, ang_df in all_stats.items():
                subdf = ang_df.loc[(ang_df["site_ce"]=="O:6") 
                       & (ang_df["site_to_ce"]=="O:6")
                       & (ang_df["site_is_tm"]) 
                       & (ang_df["site_to_is_tm"])
                ]
                if "oxygen" in data_string:
                    subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
                if not subdf.empty:
                    entries += 1
                    n_lattice_points = df.at[md_id, "n_lattice_points"]
                    occus = get_bond_angle_occurrences(df=subdf,
                                                                     include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                     normalize=normalize_bool,
                                                                     n_lattice_points=n_lattice_points,
                                                                     spin_angle_round=-1,  # for overview
                                                                     bond_angle_round=7)
                    all_spin_occus.extend(occus)
            all_spin_occus_df = pd.DataFrame(columns=["spin_angle", "bond_angle", "occurrence"], data=all_spin_occus)
            
            # Create and save one-dimensional histograms of bond angle occurrences as f(spin angle)
            one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (deg)"),
                                                   yaxis=go.layout.YAxis(title="Occurrence"),
                                                   title=f"{data_string} ({entries} structures, {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}"))
            
            for spin_ang in sorted(set(all_spin_occus_df["spin_angle"].values)):
                sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"]==spin_ang]
                
                one_d_fig.add_trace(go.Histogram(
                    histfunc="sum", 
                    x=sub_df["bond_angle"].values, 
                    y=sub_df["occurrence"].values, 
                    nbinsx=181,
                    name=spin_ang
                ))
            one_d_fig = pretty_plot(one_d_fig)
            one_d_fig.update_layout(legend_title_text="Spin angle (deg)")
            one_d_fig.update_layout(barmode="overlay")
            one_d_fig.update_traces(opacity=0.75)
            with open(one_d_hist_file, write_mode) as f:
                f.write(one_d_fig.to_html(full_html=False, include_plotlyjs="cdn"))

            # Create and save one-dimensional histograms of spin angle occurrences as f(bond angle)
            one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Spin angle (deg)"),
                                                   yaxis=go.layout.YAxis(title="Occurrence"),
                                                   title=f"{data_string} ({entries} structures, {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}",
                                                   colorway=px.colors.qualitative.Alphabet
                                                  ),
                                 )
            
            for bond_ang in sorted(set([round(ba, -1) for ba in all_spin_occus_df["bond_angle"].values])):
                sub_df = all_spin_occus_df.loc[round(all_spin_occus_df["bond_angle"], -1) == bond_ang]
                
                one_d_fig.add_trace(go.Histogram(
                    histfunc="sum", 
                    x=sub_df["spin_angle"].values, 
                    y=sub_df["occurrence"].values, 
                    nbinsx=19,
                    name=bond_ang
                ))
            one_d_fig = pretty_plot(one_d_fig)
            one_d_fig.update_layout(legend_title_text="Bond angle (deg)")
            one_d_fig.update_traces(opacity=0.75)
            with open(one_d_hist_inverse_file, write_mode) as f:
                f.write(one_d_fig.to_html(full_html=False, include_plotlyjs="cdn"))
            
            # Create and save 2D histogram of spin and bond angle occurrences
            two_d_fig = go.Figure(data=go.Histogram2d(
                x=all_spin_occus_df["bond_angle"].values,
                y=all_spin_occus_df["spin_angle"].values,
                z=all_spin_occus_df["occurrence"].values,
                histfunc="sum",  
                nbinsx=181, 
                nbinsy=19
                ),
                layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (deg)"),
                                 yaxis=go.layout.YAxis(title="Spin angle (deg)"),
                                 title=f"{data_string} ({entries} structures, {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}")
            )
            # two_d_fig.show()
            with open(two_d_hist_file, write_mode) as f:
                f.write(two_d_fig.to_html(full_html=False, include_plotlyjs="cdn"))
            write_mode = "a"

In [6]:
# Save special cases to pdf (FM and AFM subset)
normalize_bool = False
normalize_string = "absolute occurrences"
ligand_multiplicity_bool = False
ligand_multiplicity_string = "no ligand multiplicity included"

for data_string in description:
    entries = 0
    almost_collinear_entries = 0
    all_spin_occus = []
    for md_id, ang_df in all_stats.items():
        subdf = ang_df.loc[(ang_df["site_ce"]=="O:6") 
               & (ang_df["site_to_ce"]=="O:6")
               & (ang_df["site_is_tm"]) 
               & (ang_df["site_to_is_tm"])
        ]
        if "oxygen" in data_string:
            subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
        if not subdf.empty:
            entries += 1
            n_lattice_points = df.at[md_id, "n_lattice_points"]
            occus = get_bond_angle_occurrences(df=subdf,
                                                             include_ligand_multiplicity=ligand_multiplicity_bool,
                                                             normalize=normalize_bool,
                                                             n_lattice_points=n_lattice_points,
                                                             spin_angle_round=0,
                                                             bond_angle_round=7)
            all_spin_occus.extend(occus)
            if any([True for e in occus if (e[0] >= 170.0 or e[0] <= 10.0)]):
                almost_collinear_entries += 1
    all_spin_occus_df = pd.DataFrame(columns=["spin_angle", "bond_angle", "occurrence"], data=all_spin_occus)
    
    for spin_ang_lim in [10.0, 170.0]:
        if spin_ang_lim == 10.0:
            sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] <= spin_ang_lim]
        else:
            sub_df = all_spin_occus_df.loc[all_spin_occus_df["spin_angle"] >= spin_ang_lim]

        spin_ang_lim_string = f">=_{spin_ang_lim}_spin_angle" if spin_ang_lim == 170.0 else f"<=_{spin_ang_lim}_spin_angle"

        one_d_fig = go.Figure(layout=go.Layout(xaxis=go.layout.XAxis(title="Bond angle (°)"),
                                       yaxis=go.layout.YAxis(title="Occurrence"),
                                       title=f"{data_string} ({entries} structures ({almost_collinear_entries} almost coll.), {np.sum(sub_df.occurrence.values)} / {np.sum(all_spin_occus_df.occurrence.values)} occurrences), {ligand_multiplicity_string}, {normalize_string}, spin_ang_lim = {spin_ang_lim}°"))


        one_d_fig.add_trace(go.Histogram(
            histfunc="sum",
            x=sub_df["bond_angle"].values,
            y=sub_df["occurrence"].values,
            nbinsx=181,
            name=spin_ang_lim,
            marker_color="#025268"
        ))
        one_d_fig = pretty_plot(one_d_fig)
        one_d_fig.update_layout(legend_title_text="Spin angle (°)",  titlefont=dict(size=9))
        if not "oxygen" in data_string:
            one_d_fig.update_layout(yaxis_range=[0, 450], xaxis_range=[57, 182])
        else:
            one_d_fig.update_layout(yaxis_range=[0, 290], xaxis_range=[70, 182])
        one_d_fig.update_yaxes(zeroline=False)
        one_d_fig.write_image(f"plots/TM_octahedra_analysis_include_crystallographic_multiples/{data_string}_{normalize_string}_{ligand_multiplicity_string}_{spin_ang_lim_string}.pdf")

In [7]:
# KS test including crystallographic multiples w. 10 deg angle tol between neighboring magnetic vectors
for normalize_bool, normalize_string in zip([True, False], ["normalized occurrences", "absolute occurrences"]):
    for ligand_multiplicity_bool, ligand_multiplicity_string in zip([True, False], ["ligand multiplicity included", "no ligand multiplicity included"]):
        for data_string in description:
            afm_or_fm_entries = 0
            all_spin_occus = []
            for md_id, ang_df in all_stats.items():
                subdf = ang_df.loc[(ang_df["site_ce"]=="O:6")
                       & (ang_df["site_to_ce"]=="O:6")
                       & (ang_df["site_is_tm"])
                       & (ang_df["site_to_is_tm"])
                ]
                if "oxygen" in data_string:
                    subdf = subdf.loc[subdf["ligand_el_set"]=={"O"}]
                if not subdf.empty:
                    n_lattice_points = df.at[md_id, "n_lattice_points"]
                    occus = get_bond_angle_occurrences(df=subdf,
                                                                     include_ligand_multiplicity=ligand_multiplicity_bool,
                                                                     normalize=normalize_bool,
                                                                     n_lattice_points=n_lattice_points,
                                                                     spin_angle_round=0,
                                                                     bond_angle_round=7)
                    sp_a = [ls[0] for ls in occus]
                    if any([v <= 10.0 for v in sp_a]) or any([v >= 170.0 for v in sp_a]):
                        afm_or_fm_entries += 1
                    all_spin_occus.extend(occus)
            print("------")
            print(data_string, normalize_string, ligand_multiplicity_string)
            print(f"{afm_or_fm_entries} structures with FM and / or AFM interactions (10 deg angle tol).")

            fm_occus = [ls for ls in all_spin_occus if ls[0] <= 10.0]
            afm_occus = [ls for ls in all_spin_occus if ls[0] >= 170.0]
            print("d, p: ", compute_ks_weighted(afm_occus, fm_occus))

------
all edges with TM octahedra at both nodes normalized occurrences ligand multiplicity included
513 structures with FM and / or AFM interactions (10 deg angle tol).
d, p:  (0.36463976805499215, 3.863641802331604e-13)
------
all oxygen edges with TM octahedra at both nodes normalized occurrences ligand multiplicity included
391 structures with FM and / or AFM interactions (10 deg angle tol).
d, p:  (0.32586032788034847, 7.10081604922836e-08)
------
all edges with TM octahedra at both nodes normalized occurrences no ligand multiplicity included
513 structures with FM and / or AFM interactions (10 deg angle tol).
d, p:  (0.36509696299116506, 6.210477826651713e-13)
------
all oxygen edges with TM octahedra at both nodes normalized occurrences no ligand multiplicity included
391 structures with FM and / or AFM interactions (10 deg angle tol).
d, p:  (0.32355049787877477, 1.396031595581634e-07)
------
all edges with TM octahedra at both nodes absolute occurrences ligand multiplicity inc